In [11]:
import pandas as pd
from ray_sklearn import RayTrainNeuralNet
from ray_sklearn.models import TabNet
from torch import nn
from skorch.callbacks import LRScheduler
import torch
from ranger21.ranger21 import Ranger21

In [12]:
data = pd.read_csv("train 2.csv")
target = data["hand"]
data = data.drop("hand", axis=1)-1
cat_cols = data.shape[1]

In [13]:
num_cols = pd.concat([data[col] for col in data.columns if col.startswith("C")], axis=1)
num_cols.columns = [f"n{col}" for col in num_cols.columns]
data = pd.concat((data, num_cols),axis=1)

In [16]:
reg = RayTrainNeuralNet(
    TabNet,
    criterion=nn.CrossEntropyLoss,
    optimizer=torch.optim.Adam,
    callbacks=[LRScheduler(torch.optim.lr_scheduler.ReduceLROnPlateau)],
    num_workers=1,
    max_epochs=50,
    batch_size=512,
    lr=0.02,
    device="cpu",
    module__input_dim=data.shape[1],
    module__output_dim=10,
    module__cat_idxs=list(range(cat_cols)),
    module__cat_dims=[data[data.columns[i]].max()+1 for i in range(cat_cols)],
    module__cat_emb_dim=[4, 7] * (cat_cols//2),
    module__n_d=16, 
    module__n_a=8,
    module__n_steps=1,
    #optimizer__num_batches_per_epoch=data.shape[0]//512,
    #optimizer__num_epochs=50,
    iterator_train__unsqueeze_label_tensor=False,
    iterator_valid__unsqueeze_label_tensor=False,
)

In [17]:
reg.fit(data, target)

2021-12-14 00:06:52,663	INFO trainer.py:172 -- Trainer logs will be logged in: /home/ubuntu/ray_results/train_2021-12-14_00-06-52
2021-12-14 00:06:54,114	INFO trainer.py:178 -- Run results will be logged in: /home/ubuntu/ray_results/train_2021-12-14_00-06-52/run_001
(BaseWorkerMixin pid=1385158) 2021-12-14 00:06:54,109	INFO torch.py:66 -- Setting up process group for: env:// [rank=0, world_size=1]
(BaseWorkerMixin pid=1385158) 2021-12-14 00:06:54,875	INFO torch.py:239 -- Moving model to device: cpu


  epoch    train_loss    valid_loss    dur_s
-------  ------------  ------------  -------
      1        1.2382        1.0172   2.4004
      2        0.9908        1.0089   0.9257
      3        0.9853        1.0056   1.0376
      4        0.9812        1.0023   1.1683
      5        0.9782        1.0001   0.9026
      6        0.9752        0.9954   1.2763
      7        0.9681        0.9983   1.3325
      8        0.9684        0.9921   1.2943
      9        0.9654        1.0031   1.2101
     10        0.9556        0.9775   1.2598
     11        0.9392        0.9795   1.2039
     12        0.9304        0.9680   1.2781
     13        0.9211        0.9523   1.0703
     14        0.9052        0.9424   1.0538
     15        0.8934        0.9255   1.0067
     16        0.8755        0.9220   0.9895
     17        0.8661        0.9039   1.0665
     18        0.8520        0.8934   0.9974
     19        0.8327        0.8709   1.2918
     20        0.8106        0.8485   1.0149
     21   

<class 'ray_sklearn.base.RayTrainNeuralNet'>[initialized](
  module_=TabNet(
    (embedder): EmbeddingGenerator(
      (embeddings): ModuleList(
        (0): Embedding(4, 4)
        (1): Embedding(13, 7)
        (2): Embedding(4, 4)
        (3): Embedding(13, 7)
        (4): Embedding(4, 4)
        (5): Embedding(13, 7)
        (6): Embedding(4, 4)
        (7): Embedding(13, 7)
        (8): Embedding(4, 4)
        (9): Embedding(13, 7)
      )
    )
    (tabnet): TabNetNoEmbeddings(
      (initial_bn): BatchNorm1d(60, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
      (encoder): TabNetEncoder(
        (initial_bn): BatchNorm1d(60, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
        (initial_splitter): FeatTransformer(
          (shared): GLU_Block(
            (shared_layers): ModuleList(
              (0): Linear(in_features=60, out_features=48, bias=False)
              (1): Linear(in_features=24, out_features=48, bias=False)
            )
   